### Importing packages

In [1]:
# ! pip install -U pandas lxml networkx

In [2]:
import os
import sys
import re
import pickle
import json

from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET
from lxml import etree
from bs4 import BeautifulSoup as bs
from bs4 import NavigableString

#sys.path.insert(1, '../scripts/utils/')
#from text_filter import spacify_soup

In [3]:
path = '/home/jovyan/shared/C_amc_141/R_amc_3.1_12921/203_vert_spacy_rftt/'

### Retrieving pickle

In [4]:
df_files = pd.read_pickle('../outputs/df_files.pkl')

In [5]:
df_files.head()

,path,name,extension,size,atime,mtime,ctime,folder,depth,parent,uid,main,source,year,date
0,artikel_/APA/1986/APA_19860220.xml,APA_19860220,xml,6142251,2020-04-05 18:21:09,2019-09-20 08:04:05,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-02-20
1,artikel_/APA/1986/APA_19860102.xml,APA_19860102,xml,3397140,2020-02-13 14:35:33,2019-09-20 08:04:05,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-01-02
2,artikel_/APA/1986/APA_19860222.xml,APA_19860222,xml,3352934,2020-02-13 14:35:33,2019-09-20 08:04:05,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-02-22
3,artikel_/APA/1986/APA_19860103.xml,APA_19860103,xml,3794819,2020-02-13 14:35:33,2019-09-20 08:04:06,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-01-03
4,artikel_/APA/1986/APA_19860221.xml,APA_19860221,xml,5763730,2020-02-13 14:35:33,2019-09-20 08:04:06,2019-09-23 11:43:22,False,4,5,1005,artikel_,APA,1986,1986-02-21


### Examining a single file:

In [6]:
filepath = os.path.join(path, df_files.loc[0, 'path'])
print(filepath)

/home/jovyan/shared/C_amc_141/R_amc_3.1_12921/203_vert_spacy_rftt/artikel_/APA/1986/APA_19860220.xml


In [7]:
with open(filepath, "r") as f:
    soup = bs(f, "lxml-xml")

In [8]:
headers = soup.find_all('doc')
dict_headers = defaultdict(list)
list_keys = ['autor', 
             'region', 
             'datum', 
             'ressort2', 
             'tokens', 
             'keys', 
             'id', 
             'mediatype', 
             'datum_full', 
             'docsrc', 
             'docsrc_name', 
             'bibl', 
             'dupl']

for header in headers:
    for key in list_keys:
        if key in header.attrs.keys():
            dict_headers[key].append(header[key])
        else:
            dict_headers[key].append(None)
df_headers = pd.DataFrame(dict_headers)
df_headers.head()

,autor,region,datum,ressort2,tokens,keys,id,mediatype,datum_full,docsrc,docsrc_name,bibl,dupl
0,wm,agesamt,1986-02-20,ausland chronik,292,JA_1986 AG_APA RS_CA RS_C DA_19860220 MO_19860...,APA_19860220_APA0001,print,1986-02-20T00:00:00Z,APA,APA-Meldungen digital,APA-Meldungen digital vom 1986-02-20,None
1,wm,agesamt,1986-02-20,ausland kultur,322,JA_1986 AG_APA RS_KA RS_K DA_19860220 MO_19860...,APA_19860220_APA0002,print,1986-02-20T00:00:00Z,APA,APA-Meldungen digital,APA-Meldungen digital vom 1986-02-20,None
2,wm,agesamt,1986-02-20,ausland kultur,332,JA_1986 AG_APA RS_KA RS_K DA_19860220 MO_19860...,APA_19860220_APA0003,print,1986-02-20T00:00:00Z,APA,APA-Meldungen digital,APA-Meldungen digital vom 1986-02-20,None
3,hs,agesamt,1986-02-20,ausland inland medien sport,146,JA_1986 AG_APA RS_AA RS_A DA_19860220 MO_19860...,APA_19860220_APA0004,print,1986-02-20T00:00:00Z,APA,APA-Meldungen digital,APA-Meldungen digital vom 1986-02-20,None
4,awi hr,agesamt,1986-02-20,ausland,168,JA_1986 AG_APA RS_AA RS_A DA_19860220 MO_19860...,APA_19860220_APA0005,print,1986-02-20T00:00:00Z,APA,APA-Meldungen digital,APA-Meldungen digital vom 1986-02-20,None


In [9]:
docs = soup.find_all('field')
print("number of docs: ", len(docs))

#print('-----')
for doc in docs[0:5]:
    for key in doc.attrs.keys():
        print(f"{key}: {doc[key]}")
print('-----')

stichwort = soup.find_all(attrs={"name" : "stichwort"})
print(f'{len(stichwort)} stichwort elements')
titel = soup.find_all(attrs={"name" : "titel"})
print(f'{len(titel)} titel elements')
inhalt = soup.find_all(attrs={"name" : "inhalt"})
print(f'{len(inhalt)} inhalt elements')

number of docs:  1448
name: stichwort
name: stichwort
name: stichwort
name: titel
name: inhalt
-----
796 stichwort elements
326 titel elements
326 inhalt elements


In [10]:
for doc in docs[0:5]:
    print(f"name: {doc['name']}")
    for d in doc.find_all('s'):
        text = re.sub('<g/>','',d.text)
        text = re.sub(r'[\n]+','\n',text)
        rows = re.split('\n',text)[1:-1]
        for row in rows:
            fields = re.split('\t',row)
            print(len(fields[:-1]), fields[:-1])
    print('------------')

name: stichwort
15 ['Umwelt', 'umwelt', '0', 'NOUN', 'NN', '', 'O', '0', 'ROOT', '0', '-', 'Umwelt', 'N.Reg.Nom.Sg.Fem', 'NN', 'Umwelt-n']
------------
name: stichwort
15 ['USA', 'usa', '0', 'PROPN', 'NE', 'LOC', 'B', '0', 'ROOT', '0', '-', '-', 'N.Name.Gen.Sg.*', 'NE', 'USA-n']
------------
name: stichwort
15 ['F', 'f', '0', 'X', 'FM', '', 'O', '-', 'ROOT', '0', '-', '-', 'N.Reg.Nom.Sg.Neut', 'NN', 'F-n']
15 ['e', 'e', '1', 'X', 'FM', '', 'O', '-', 'uc', '0', '-', '-', 'FM', 'ADJA', 'e-n']
15 ['a', 'a', '2', 'X', 'FM', '', 'O', '-', 'uc', '0', '-', '-', 'FM', 'FM', 'a-x']
15 ['t', 't', '3', 'X', 'FM', '', 'O', '-', 'uc', '0', '-', '-', 'FM', 'NN', 't-n']
15 ['u', 'u', '4', 'X', 'FM', '', 'O', '-', 'uc', '0', '-', '-', 'N.Name.Nom.Sg.Neut', 'ADJA', 'U-n']
15 ['r', 'r', '5', 'X', 'FM', '', 'O', '-', 'uc', '0', '-', '-', 'N.Name.Nom.Sg.Neut', 'ADJA', 'R-n']
15 ['e', 'e', '6', 'X', 'FM', '', 'O', '-', 'uc', '0', '-', '-', 'N.Name.*.*.*', 'NN', 'E-n']
------------
name: titel
15 ['Geht', '